Sources:
- Data: https://agupubs.onlinelibrary.wiley.com/doi/full/10.1029/2019RG000691
- The original code for the evaluation of GWP/GTP from AR6 Table 7.SM.7 is available here: https://github.com/chrisroadmap/ar6/blob/main/notebooks/335_chapter7_generate_metrics.ipynb
- The actual metrics calculations is here: https://github.com/chrisroadmap/ar6/tree/main/src/ar6/metrics

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('/Users/timodiepers/Documents/Coding/timex/timex_lca/data/hodnebrog20.csv', usecols=["CASRN", "Molar mass", "Lifetime (yr)", "RE (W m-2 ppb-1)"]).dropna()

In [3]:
data

,CASRN,Molar mass,Lifetime (yr),RE (W m-2 ppb-1)
1,75-69-4,0.13736,52.0000,0.25941
2,75-71-8,0.12091,102.0000,0.31998
3,75-72-9,0.10446,640.0000,0.27752
4,76-12-0,0.20382,63.6000,0.28192
5,76-11-9,0.20382,52.0000,0.24564
...,...,...,...,...
610,141-63-9,0.38489,0.0110,0.06400
611,541-05-9,0.22248,0.0384,0.10000
612,556-67-2,0.29664,0.0274,0.12000
613,541-02-6,0.37080,0.0164,0.09800


In [4]:
def format_cas(cas):
    # Split the CAS number by the dash
    parts = cas.split('-')
    # Format the first part to ensure it has 6 digits
    parts[0] = parts[0].zfill(6)
    # Join the parts back together and return
    return '-'.join(parts)

M_air = 28.97e-3  # kg/mol, dry air
m_atmosphere = 5.135e18  # kg [Trenberth and Smith, 2005]

decay_multipliers_dict = {}
 
for cas, tau, M, radiative_efficiency_ppb in zip(data['CASRN'], data['Lifetime (yr)'], data['Molar mass'], data['RE (W m-2 ppb-1)']):
    
    if cas == "2551-62-4":
        tau = 1000 # Decreased lifetime of SF6 from 3200 to 1000 years. See IPCC AR6, Ch. 2.2.4.3. doi: 10.1017/9781009157896.004
    
    # for conversion from ppb to kg-CH4
    radiative_efficiency_kg = (
        radiative_efficiency_ppb * M_air / M * 1e9 / m_atmosphere
    )  # W/m2/kg-CH4;

    decay_multipliers: list = np.array(
        [
            radiative_efficiency_kg * tau * (1 - np.exp(-year / tau))
            for year in range(2000)
        ]
    ).tolist()
    
    decay_multipliers_dict[format_cas(cas)] = decay_multipliers

In [5]:
import json

In [6]:
with open('decay_multipliers.json', 'w') as f:
    json.dump(decay_multipliers_dict, f)